# Custom solvers
In this example, we show how to define custom solvers. Our system
will again be silicon, because we are not very imaginative

In [1]:
using DFTK, LinearAlgebra

a = 10.26
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]]

# We take very (very) crude parameters
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[1, 1, 1]);

We define our custom fix-point solver: simply a damped fixed-point

In [2]:
function my_fp_solver(f, x0, max_iter; tol)
    mixing_factor = .7
    x = x0
    fx = f(x)
    for n = 1:max_iter
        inc = fx - x
        if norm(inc) < tol
            break
        end
        x = x + mixing_factor * inc
        fx = f(x)
    end
    (fixpoint=x, converged=norm(fx-x) < tol)
end;

Our eigenvalue solver just forms the dense matrix and diagonalizes
it explicitly (this only works for very small systems)

In [3]:
function my_eig_solver(A, X0; maxiter, tol, kwargs...)
    n = size(X0, 2)
    A = Array(A)
    E = eigen(A)
    λ = E.values[1:n]
    X = E.vectors[:, 1:n]
    (λ=λ, X=X, residual_norms=[], iterations=0, converged=true, n_matvec=0)
end;

Finally we also define our custom mixing scheme. It will be a mixture
of simple mixing (for the first 2 steps) and than default to Kerker mixing.
In the mixing interface `δF` is ``(ρ_\text{out} - ρ_\text{in})``, i.e.
the difference in density between two subsequent SCF steps and the `mix`
function returns ``δρ``, which is added to ``ρ_\text{in}`` to yield ``ρ_\text{next}``,
the density for the next SCF step.

In [4]:
struct MyMixing
    n_simple  # Number of iterations for simple mixing
end
MyMixing() = MyMixing(2)

function DFTK.mix_density(mixing::MyMixing, basis, δF; n_iter, kwargs...)
    if n_iter <= mixing.n_simple
        return δF  # Simple mixing -> Do not modify update at all
    else
        # Use the default KerkerMixing from DFTK
        DFTK.mix_density(KerkerMixing(), basis, δF; kwargs...)
    end
end

That's it! Now we just run the SCF with these solvers

In [5]:
scfres = self_consistent_field(basis;
                               tol=1e-8,
                               solver=my_fp_solver,
                               eigensolver=my_eig_solver,
                               mixing=MyMixing());

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -7.227971221753         NaN   3.22e-01   0.80    0.0
  2   -7.246548565129   -1.86e-02   1.26e-01   0.80    0.0
  3   -7.248998824732   -2.45e-03   4.70e-02   0.80    0.0
  4   -7.249162119311   -1.63e-04   2.30e-02   0.80    0.0
  5   -7.249202768478   -4.06e-05   1.14e-02   0.80    0.0
  6   -7.249213089302   -1.03e-05   5.74e-03   0.80    0.0
  7   -7.249215815319   -2.73e-06   2.95e-03   0.80    0.0
  8   -7.249216572087   -7.57e-07   1.54e-03   0.80    0.0
  9   -7.249216793514   -2.21e-07   8.23e-04   0.80    0.0
 10   -7.249216861576   -6.81e-08   4.48e-04   0.80    0.0
 11   -7.249216883397   -2.18e-08   2.48e-04   0.80    0.0
 12   -7.249216890631   -7.23e-09   1.40e-04   0.80    0.0


Note that the default convergence criterion is on the difference of
energy from one step to the other; when this gets below `tol`, the
"driver" `self_consistent_field` artificially makes the fixpoint
solver think it's converged by forcing `f(x) = x`. You can customize
this with the `is_converged` keyword argument to
`self_consistent_field`.